# Импорты

In [ ]:
!pip install pymorphy3 nltk -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 41.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

from IPython.display import Image
from IPython.core.display import HTML
%matplotlib inline

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"
# os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
torch.set_default_device('cuda')

import keras
print(keras.__version__)

3.5.0


Домашку будет легче делать в колабе (убедитесь, что у вас runtype с gpu).

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты:
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна в большую или меньшую сторону
3) измените размерность итоговых векторов

Выберете несколько не похожих по смыслу слов (не таких как в семинаре), и протестируйте полученные эмбединги (найдите ближайшие слова и оцените качество, как в семинаре).
Постарайтесь обучать модели как можно дольше и на как можно большем количестве данных. (Но если у вас мало времени или ресурсов, то допустимо взять поменьше данных и поставить меньше эпох)

## Предобработка и создание словарей

In [ ]:
wiki = open('wiki_data.txt').read().split('\n')

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import pymorphy3

nltk.download('punkt_tab')

def preprocess(text):
    tokens = re.sub('#+', ' ', text.lower())
    tokens = word_tokenize(text)
    morph = pymorphy3.MorphAnalyzer()

    processed_tokens = []
    for token in tokens:
        if token and token not in punctuation:
            processed_tokens.append(morph.parse(token)[0].normal_form)

    return processed_tokens

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Я один раз создала словарь и сохранила его к себе на диск, но постоянно забывала про это и запускала ячейки, которые уже не нужно запускать( Промежуточные файлы в репозитории

In [ ]:
from collections import Counter
from tqdm import tqdm

vocab = Counter()

for text in tqdm(wiki):
    vocab.update(preprocess(text))

  0%|          | 35/20003 [00:03<35:35,  9.35it/s]


KeyboardInterrupt: 

In [1]:
from google.colab import drive
import pickle
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/unfiltered_vocab.pkl", 'wb') as f:
    pickle.dump(vocab, f)

In [ ]:
filtered_vocab = set()

for word in tqdm(vocab):
    if vocab[word] > 30:
        filtered_vocab.add(word)

100%|██████████| 160489/160489 [00:00<00:00, 1241261.17it/s]


In [ ]:
with open("/content/drive/MyDrive/filtered_vocab.pkl", 'wb') as f:
    pickle.dump(filtered_vocab, f)

In [2]:
with open("/content/drive/MyDrive/filtered_vocab.pkl", 'rb') as f:
    filtered_vocab = pickle.load(f)

In [3]:
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [4]:
id2word = {i:word for word, i in word2id.items()}

In [ ]:
from tqdm import tqdm

In [ ]:
text_sentences = [] # сразу сохранила предобработанные тексты для второго задания
sentences = []


for text in tqdm(wiki):
    tokens = preprocess(text)
    text_sentences.append(tokens)
    if not tokens:
        continue
    ids = [word2id[token] for token in tokens if token in word2id]
    sentences.append(ids)

100%|██████████| 20003/20003 [35:40<00:00,  9.34it/s]


In [ ]:
with open("/content/drive/MyDrive/wiki_sentences.pkl", "wb") as f:
    pickle.dump(sentences, f)

In [ ]:
with open("/content/drive/MyDrive/wiki_text_sentences.pkl", "wb") as f:
    pickle.dump(text_sentences, f)

In [ ]:
with open("/content/drive/MyDrive/wiki_sentences.pkl", "rb") as f:
    sentences = pickle.load(f)

In [ ]:
with open("/content/drive/MyDrive/wiki_text_sentences.pkl", "rb") as f:
    text_sentences = pickle.load(f)

In [5]:
vocab_size = len(id2word)

## Negative sampling

In [ ]:
# skip gram
def gen_batches_sg(sentences, window = 7, batch_size=1000):

    left_context_length = (window/2).__ceil__() # округлить в большую сторону
    right_context_length = window // 2 # округлить в меньшую сторону

    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-left_context_length):i] + sent[i+1:i+right_context_length]
                for context_word in context:
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)

                    X_target.append(word)
                    X_context.append(np.random.randint(vocab_size))
                    y.append(0)

                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []

In [ ]:
def gen_batches_cbow(sentences, window = 7, batch_size=1000):

    left_context_length = (window/2).__ceil__() # округлить в большую сторону
    right_context_length = window // 2 # округлить в меньшую сторону

    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-left_context_length):i] + sent[i+1:i+right_context_length]

                X_target.append(word)
                X_context.append(context)
                y.append(1)

                X_target.append(np.random.randint(vocab_size))
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = keras.preprocessing.sequence.pad_sequences(X_context, maxlen=window)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

## Обучение

In [ ]:
inputs_target = keras.layers.Input(shape=(1,))
inputs_context = keras.layers.Input(shape=(1,))


embeddings_target = keras.layers.Embedding(input_dim=len(word2id), output_dim=500)(inputs_target, )
embeddings_context = keras.layers.Embedding(input_dim=len(word2id), output_dim=500)(inputs_context, )

target = keras.layers.Flatten()(embeddings_target)
context = keras.layers.Flatten()(embeddings_context)

dot = keras.layers.Dot(1)([target, context])
outputs = keras.layers.Activation(activation='sigmoid')(dot)

model = keras.Model(inputs=[inputs_target, inputs_context],
                       outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(gen_batches_sg(sentences, window=7),
          validation_data=gen_batches_sg(sentences,  window=7),
          batch_size=1000,
          steps_per_epoch=10000,
          validation_steps=30,
          epochs=20)

model.save('/content/drive/MyDrive/wiki_emb_model.keras')

Epoch 1/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 142s 14ms/step - accuracy: 0.7747 - loss: 0.4781 - val_accuracy: 0.7904 - val_loss: 0.4746
Epoch 2/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 135s 13ms/step - accuracy: 0.8121 - loss: 0.4224 - val_accuracy: 0.7832 - val_loss: 0.5162
Epoch 3/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 137s 14ms/step - accuracy: 0.8199 - loss: 0.4052 - val_accuracy: 0.8259 - val_loss: 0.3873
Epoch 4/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 134s 13ms/step - accuracy: 0.8241 - loss: 0.3940 - val_accuracy: 0.8397 - val_loss: 0.3708
Epoch 5/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 136s 14ms/step - accuracy: 0.8255 - loss: 0.3916 - val_accuracy: 0.8228 - val_loss: 0.4025
Epoch 6/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 135s 14ms/step - accuracy: 0.8242 - loss: 0.3965 - val_accuracy: 0.8194 - val_loss: 0.3939
Epoch 7/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 135s 14ms/step - accuracy: 0.8247 - loss: 0.3959 - val_accuracy: 0.8361 - val_loss: 0.3681
Epoch 8/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 136s 14ms/s

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/wiki_emb_model.keras')

In [ ]:
embeddings = model.layers[2].get_weights()[0]

In [ ]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

In [ ]:
most_similar('компьютер', embeddings)

['компьютер',
 'мур',
 'уметь',
 'штемпель',
 'убийство',
 'момент',
 'сидение',
 'работа',
 'пропаганда',
 'переехать']

# Задание 2 (2 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [ ]:
import gensim

In [ ]:
w2v = gensim.models.Word2Vec(text_sentences,
                             vector_size=256,
                             min_count=50,
                             max_vocab_size=8000,
                             window=7,
                             epochs=5,
                             hs=0,
                             negative=6,
                             sample=1e-4,
                             ns_exponent=0.3
                            )

In [ ]:
ft = gensim.models.FastText(text_sentences,
                            min_n=4,
                            max_n=7,
                            vector_size=256,
                             min_count=50,
                             max_vocab_size=8000,
                             window=7,
                             epochs=5,
                             hs=0,
                             negative=6,
                             sample=1e-4,
                             ns_exponent=0.3
                            )

In [ ]:
w2v.wv.most_similar('церковь')

[('храм', 0.871390700340271),
 ('часовня', 0.8643491864204407),
 ('православный', 0.8577724695205688),
 ('монастырь', 0.8445532321929932),
 ('приход', 0.832804262638092),
 ('собор', 0.8215623497962952),
 ('икона', 0.8138442039489746),
 ('церковный', 0.8015328645706177),
 ('епархия', 0.785042941570282),
 ('святой', 0.7434539794921875)]

In [ ]:
ft.wv.most_similar('церковь')

[('монастырь', 0.9101859331130981),
 ('собор', 0.8775367736816406),
 ('храм', 0.8688487410545349),
 ('церковный', 0.860481858253479),
 ('часовня', 0.8480122685432434),
 ('епархия', 0.825242280960083),
 ('православный', 0.8159268498420715),
 ('приход', 0.8103792071342468),
 ('икона', 0.8016388416290283),
 ('святой', 0.7823973894119263)]

# Задание 3 (3 балла)

Используя датасет для классификации (labeled.csv), обучите классификатор на базе эмбеддингов. Оцените качество на отложенной выборке.   
В качестве эмбеддинг модели вы можете использовать одну из моделей обученных в предыдущем задании или использовать одну из предобученных моделей с rusvectores (удостоверьтесь что правильно воспроизводите предобработку в этом случае!)  
Для того, чтобы построить эмбединг целого текста, усредните вектора отдельных слов в один общий вектор.
В качестве алгоритма классификации используйте LogisicticRegression (можете попробовать SGDClassifier, чтобы было побыстрее)  
F1 мера должна быть выше 20%.

Написала функцию для векторизации и функцию, в которую завёрнута функция предобработки и векторизации, чтобы было удобнее и быстрее обрабатывать датасет

In [ ]:
def vectorize_sentence(sentence, fasttext_model):
  word_vectors = []
  for word in sentence:
      try:
          word_vectors.append(fasttext_model.wv[word])
      except KeyError:
          continue

  if not word_vectors:
        return np.zeros(fasttext_model.vector_size)

  sentence_vector = np.mean(word_vectors, axis=0)
  return sentence_vector

In [ ]:
def df_preprocess(text, text_preproc_func=preprocess, vect_func=vectorize_sentence, emb_model=w2v):
  processed = text_preproc_func(text)
  vectorized = vect_func(processed, emb_model)
  return vectorized

In [ ]:
df = pd.read_csv("labeled.csv")
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
df_comm = df["comment"].progress_apply(df_preprocess)
df_comm

  0%|          | 0/14412 [00:00<?, ?it/s]

,comment
0,"[-0.5479859, -0.115324885, -0.1439715, 0.54171..."
1,"[-0.31704652, -0.0266949, 0.3103802, 0.3094891..."
2,"[-0.2878002, -0.50649345, 0.12595865, 0.092326..."
3,"[-0.23058704, -0.09230675, 0.38427347, 0.33193..."
4,"[-0.38383955, -0.19533716, 0.28752023, 0.29085..."
...,...
14407,"[-0.16469301, -0.0854789, 0.29672965, 0.212611..."
14408,"[-0.014553403, -0.17680919, 0.6171532, 0.19577..."
14409,"[-0.11290816, -0.100725256, 0.17473896, 0.4153..."
14410,"[-0.07155909, -0.1602273, 0.35228077, 0.220061..."


In [ ]:
df_comm.to_csv("labeled_vec.csv")

In [ ]:
df.head(5)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [ ]:
X = df_comm.to_list()
y = df["toxic"].to_list()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
clf = LogisticRegression(random_state=42).fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

In [ ]:
f1

0.7531804950674202

In [ ]:
accuracy

0.7662157474852584

In [ ]:
precision

0.7598020294955304

In [ ]:
recall

0.7662157474852584